Nama : Annisa Putri Aulia

Nim : H071191029

Tugas Deep Learning Minggu 3

soal : Pelajari https://d2l.ai/chapter_deep-learning-computation/index.html

In [1]:
import tensorflow as tf
from tensorflow import keras

1. Layer dan Blocks
Layer merupakan arsitektur atau struktur yang berisi kumpulan node yang beroperasi secara dan di waktu yang bersamaan, yang mengambil informasi dari layer sebelumnya dan kemudian meneruskan informasi ke layer berikutnya.

Block atau neural network blocks merupakan konsep di mana satu atau lebih layer digambarkan / diabstraksi menjadi satu komponen (block). Salah satu manfaat utama dari abstraksi blocks adalah blocks ini bisa lagi dikombinasikan menjadi struktur yang lebih besar (biasanya secara rekursif).


Dengan menggunakan konsep blocks ini, kita tetap dapat menulis kode yang ringkas dan masih mengimplementasikan neural net yang kompleks. Untuk mengimplementasikan blocks, kita membuat fungsi atau jika ingin fleksibel, kelas turunan dari tensorflow.keras.Model yang memiliki fungsi forward propagation (fungsi call()) . Contoh implementasi kode

In [2]:
# Data: digenerate secara random
X = tf.random.uniform((2, 20))

X

<tf.Tensor: shape=(2, 20), dtype=float32, numpy=
array([[0.30787158, 0.3728453 , 0.03039014, 0.60959554, 0.960413  ,
        0.29244947, 0.5469928 , 0.12978256, 0.4818523 , 0.5512279 ,
        0.681303  , 0.27853668, 0.5265553 , 0.71858525, 0.968745  ,
        0.4811046 , 0.5655756 , 0.1803906 , 0.29843092, 0.92003584],
       [0.08929896, 0.25036108, 0.67826724, 0.9433805 , 0.49905705,
        0.90702677, 0.7370039 , 0.8914101 , 0.02507567, 0.28068447,
        0.68392515, 0.34574533, 0.7581719 , 0.02627659, 0.5795628 ,
        0.76301944, 0.41747952, 0.9528055 , 0.05830562, 0.9686831 ]],
      dtype=float32)>

Misalnya model yang akan dijadikan blok merupakan model seperti ini

In [3]:
block1 = keras.models.Sequential([
    keras.layers.Dense(256, activation=tf.nn.relu),
    keras.layers.Dense(10),
])

Bentuk bloknya:

In [4]:
class Block1(keras.Model):
    def __init__(self):
        super().__init__()
        self.layer_list= []
        self.layer_list.append(keras.layers.Dense(256, activation=tf.nn.relu))
        self.layer_list.append(keras.layers.Dense(10))
        self.layer_list.reverse()
    
    def call(self, X):
        result_temp= X

        # Melakukan forward propagation sampai layer terakhir dalam list 
        while bool(self.layer_list):
            result_temp= self.layer_list.pop()(result_temp)

        return result_temp
        # return 1

In [5]:
net= Block1()
net(X) # Sama saja dengan net.call(X)

<tf.Tensor: shape=(2, 10), dtype=float32, numpy=
array([[ 0.00637328,  0.23333551, -0.07434475,  0.12067626,  0.20530316,
         0.29963946,  0.00518246, -0.07541624,  0.08535212, -0.00926593],
       [ 0.00639129,  0.12510358, -0.12528375,  0.15067631,  0.17411584,
         0.325081  , -0.19129105,  0.13203019, -0.03842275, -0.05086474]],
      dtype=float32)>

Selanjutnya kita akan membuat block ke dua yang tidak mengandung layer satu per satu tapi mengandung model

In [6]:
class Block2(keras.Model):
    def __init__(self):
        super().__init__()
        self.net= keras.Sequential()
        self.net.add(keras.layers.Dense(256, activation=tf.nn.relu))
        self.net.add(keras.layers.Dense(10))
    
    def call(self, X):
        return self.net(X)

Kemudian disatukan bersama

In [7]:
net= keras.Sequential()

# Menambahkan layer pertama yang mana merupakan block 2
net.add(Block2())

# Menambahkan block 1
net.add(Block1())

# # Menambahkan satu lagi layer dense biasa
net.add(keras.layers.Dense(10, activation= 'sigmoid'))

# Melakukan forward propagation
net(X)

<tf.Tensor: shape=(2, 10), dtype=float32, numpy=
array([[0.57707757, 0.58166116, 0.5321636 , 0.6268364 , 0.5452533 ,
        0.49243551, 0.43619287, 0.5362292 , 0.41937172, 0.46161866],
       [0.573516  , 0.5937405 , 0.5796414 , 0.56678647, 0.58025855,
        0.5279852 , 0.44973466, 0.6054406 , 0.5076211 , 0.40960175]],
      dtype=float32)>

**2. Parameter Management**
Parameter management atau manajemen parameter termasuk (Parameter dalam kasus neural network contohnya weight dan bias, fungsi loss atau learning rate merupakan hyperparameter):

Mengakses parameter untuk debugging, diagnostik, dan
visualisasi.
Inisialisasi parameter.
Berbagi parameter dengan komponen model lain.

**Mengakses Parameter**

Model:

In [8]:
net = keras.models.Sequential([
    keras.layers.Flatten(),
    keras.layers.Dense(4, activation=tf.nn.relu),
    keras.layers.Dense(1),
])

X = tf.random.uniform((2, 4))
net(X)

<tf.Tensor: shape=(2, 1), dtype=float32, numpy=
array([[-0.10075305],
       [ 0.03059036]], dtype=float32)>

Mengambil weight dan bias dari setiap layer net

In [9]:
net.weights
# Atau net.get_weights()

[<tf.Variable 'dense_9/kernel:0' shape=(4, 4) dtype=float32, numpy=
 array([[-0.49756283,  0.627852  , -0.38583094,  0.1821503 ],
        [-0.3833196 , -0.23057151, -0.06623125,  0.83617014],
        [ 0.40399784,  0.67305535, -0.4240956 ,  0.5449104 ],
        [-0.2774384 , -0.40975744,  0.63852435, -0.6100112 ]],
       dtype=float32)>,
 <tf.Variable 'dense_9/bias:0' shape=(4,) dtype=float32, numpy=array([0., 0., 0., 0.], dtype=float32)>,
 <tf.Variable 'dense_10/kernel:0' shape=(4, 1) dtype=float32, numpy=
 array([[ 0.16241813],
        [ 0.05146575],
        [ 0.7021854 ],
        [-0.2188415 ]], dtype=float32)>,
 <tf.Variable 'dense_10/bias:0' shape=(1,) dtype=float32, numpy=array([0.], dtype=float32)>]

Untuk mendapatkan parameter pada layer yang ingin diakses, dilakukan dengan memanggil atribut .layers[index dari layer] dari model sequential. Misal layer yang ingin diakses parameternya yaitu layer indeks ke-1

In [10]:
net.layers[1]

Selanjutnya untuk mengakses parameter weight dan bias

In [11]:
net.layers[1].weights

[<tf.Variable 'dense_9/kernel:0' shape=(4, 4) dtype=float32, numpy=
 array([[-0.49756283,  0.627852  , -0.38583094,  0.1821503 ],
        [-0.3833196 , -0.23057151, -0.06623125,  0.83617014],
        [ 0.40399784,  0.67305535, -0.4240956 ,  0.5449104 ],
        [-0.2774384 , -0.40975744,  0.63852435, -0.6100112 ]],
       dtype=float32)>,
 <tf.Variable 'dense_9/bias:0' shape=(4,) dtype=float32, numpy=array([0., 0., 0., 0.], dtype=float32)>]

Outputnya berupa array, indeks ke-0 merupakan weights, indeks ke-1 merupakan bias

Untuk layer yang berupa block (memiliki layer lagi di dalamnya), kita tetap bisa mengakses nya seperti ini:

net.layers[0].layers[1].layers[1].weights[1]

**Inisialisasi Parameter**

Secara default (untuk tensorflow), matriks weight diinisialisasikan berdasarkan range yang dikomputasi dari dimensi input dan output, sementara parameter bias mempunyai nilai default 0.

Misal menginisialisasi weight sebagai variabel random Gaussian dengan standar deviasi 0.01, dan menginisialisasi parameter bias dengan nilai 0

In [12]:
net = keras.models.Sequential([
    keras.layers.Flatten(),
    keras.layers.Dense(4, activation=tf.nn.relu, 
                       kernel_initializer=tf.random_normal_initializer(mean=0, stddev=0.01), 
                       bias_initializer=tf.zeros_initializer()),
    keras.layers.Dense(1)
])

net(X)
net.layers[1].weights[0], net.layers[1].weights[1]

(<tf.Variable 'dense_11/kernel:0' shape=(4, 4) dtype=float32, numpy=
 array([[-0.0019742 , -0.01118323, -0.00892487,  0.00309992],
        [-0.00017186,  0.01191985,  0.00046774, -0.00797555],
        [ 0.00871554,  0.0064146 ,  0.02402176,  0.00839928],
        [-0.01670321,  0.00804337,  0.02824247, -0.00047538]],
       dtype=float32)>,
 <tf.Variable 'dense_11/bias:0' shape=(4,) dtype=float32, numpy=array([0., 0., 0., 0.], dtype=float32)>)

Kita juga bisa menginisialisasikan weight yang berbeda pada setiap layer, misal weight layer dense kedua diisi dengan angka konstan 0.5.

In [13]:
net = keras.models.Sequential([
    keras.layers.Flatten(),
    keras.layers.Dense(4, activation=tf.nn.relu, 
                       kernel_initializer=tf.random_normal_initializer(mean=0, stddev=0.01), 
                       bias_initializer=tf.zeros_initializer()),
    keras.layers.Dense(4, activation=tf.nn.relu, 
                       kernel_initializer=keras.initializers.Constant(0.5), 
                       bias_initializer=tf.zeros_initializer()),
    keras.layers.Dense(1)
])

net(X)
net.layers[1].weights[0], net.layers[2].weights[0]

(<tf.Variable 'dense_13/kernel:0' shape=(4, 4) dtype=float32, numpy=
 array([[ 0.00741534, -0.0134807 , -0.00712053,  0.00797166],
        [-0.00052987, -0.01749855,  0.01553926,  0.0109643 ],
        [-0.00527088,  0.00283997,  0.00884217,  0.00102325],
        [ 0.00343966, -0.00746868,  0.0097209 , -0.01068751]],
       dtype=float32)>,
 <tf.Variable 'dense_14/kernel:0' shape=(4, 4) dtype=float32, numpy=
 array([[0.5, 0.5, 0.5, 0.5],
        [0.5, 0.5, 0.5, 0.5],
        [0.5, 0.5, 0.5, 0.5],
        [0.5, 0.5, 0.5, 0.5]], dtype=float32)>)

Kita juga bisa membuat subclass kita sendiri untuk inisialisasi weight jika cara di atas kurang flexibel, caranya yaitu kita membuat class subclass dari keras.initializers.Initializer , lalu mengimplementasikan method call(shape, dtype) yang me-return tensor (berisi weight inisial) yang isinya sesuai kemauan kita

In [14]:
# Di sini kita membuat class untuk inisialisasi weight
class MyInit(tf.keras.initializers.Initializer):
    def __call__(self, shape, dtype=None):
        data=tf.random.uniform(shape, -10, 10, dtype=dtype)
        factor=(tf.abs(data) >= 5)
        factor=tf.cast(factor, tf.float32)
        return data * factor

In [15]:
MyInit().__call__(shape=(3, 3), dtype=tf.float32)

<tf.Tensor: shape=(3, 3), dtype=float32, numpy=
array([[ 8.308256, -0.      , -0.      ],
       [-0.      ,  5.700405,  5.467098],
       [ 5.282655,  0.      , -0.      ]], dtype=float32)>

In [16]:
net = keras.models.Sequential([
    keras.layers.Flatten(),
    # Memasukkan kelas initializer pada parameter kernel_initializer
    keras.layers.Dense(4, activation=tf.nn.relu, kernel_initializer=MyInit()),
    keras.layers.Dense(1),
])

net(X)
print(net.layers[1].weights[0])

<tf.Variable 'dense_16/kernel:0' shape=(4, 4) dtype=float32, numpy=
array([[-9.82738  , -0.       ,  0.       ,  0.       ],
       [ 5.5889816,  0.       , -9.834247 ,  7.05093  ],
       [-0.       , -7.3816943, -0.       , -7.33598  ],
       [ 0.       ,  5.0180054, -0.       , -0.       ]], dtype=float32)>


**3. Deferred Initialization**

Inisialisasi weight pada model ditangguhkan sebelum data di-pass ke model. Ini dikarenakan model masih belum mengetahui input shape dari data, maka dari itu model menangguhkan inisialisasi parameter, konsep ini juga membuat kita tidak perlu mendefinisikan input shape dari setiap layer, yang mana pastinya sangat membantu terutama jika dataset berupa gambar (yang mana mempunyai shape yang dinamis)

Contohnya seperti kasus berikut, pertama-tama kita membuat model lalu memanggil weight nya

In [17]:
net = keras.models.Sequential([
    keras.layers.Dense(256, activation=tf.nn.relu),
    keras.layers.Dense(10),
])

In [18]:
[net.layers[i].get_weights() for i in range(len(net.layers))]

[[], []]

Kita bisa melihat weightnya masih kosong karena belum diinisialisasikan sama sekali

In [19]:
# Feed model dengan X
net(X)

[w.shape for w in net.get_weights()], net.layers[0].get_weights()[0]

([(4, 256), (256,), (256, 10), (10,)],
 array([[ 0.12502024,  0.02418718,  0.11492026, ..., -0.01663141,
          0.01777968, -0.13601425],
        [-0.01434954,  0.10520825,  0.06597562, ...,  0.05124481,
         -0.04649194,  0.00673318],
        [ 0.03853755,  0.0249256 ,  0.08270018, ...,  0.15005407,
         -0.07792979,  0.11418673],
        [ 0.14403996, -0.14539026, -0.10201874, ..., -0.06597048,
          0.02777791,  0.13900194]], dtype=float32))

Barulah setelah data di-pass weightnya terinisialisasi. Ini dinamakan Deferred Initialization (penangguhan inisialisasi).

**4. Custom Layers**

Pada kasus-kasus tertentu, kita berhadapan dengan masalah di mana kita membutuhkan layer yang belum pernah dibuat atau belum ada pada library, untuk mengatasi masalah tersebut, kita bisa membuat layer kustom kita sendiri. Membuat layer kustom sama saja seperti membuat blok. Terdapat 2 jenis kustom layer: layer tanpa atau dengan parameter (seperti weight atau bias)

**Layer tanpa parameter**

Layer tanpa parameter merupakan layer yang tidak mempunyai weight dan bias di dalamnya (seperti layer flatten). Misal kita ingin membuat layer yang mengurangi input dengan satu

In [20]:
class MinusOneLayer(keras.Model):
    def __init__(self):
        super().__init__()

    def call(self, input):
        return input - tf.ones(input.shape)

In [21]:
layer_wo_params= MinusOneLayer()
layer_wo_params(tf.constant([1.,2.,3.]))

<tf.Tensor: shape=(3,), dtype=float32, numpy=array([0., 1., 2.], dtype=float32)>

Mengimplementasikannya pada model

In [22]:
net = keras.Sequential([keras.layers.Dense(128), MinusOneLayer()])

**Layer dengan parameter**

Layer dengan parameter mempunyai dan melibatkan weight dan bias pada operasinya. Misal kita ingin membuat layer dense yang berfungsi sama dengan layer dense pada umumnya (input * weight + bias lalu aktivasi). Walaupun kita mengatur/inisialisasi parameter semau kita, parameter juga tetap berubah saat proses back propagation

In [23]:
class MyDense(keras.Model):
    # Di sini kita berikan dua parameter saja yaitu untuk output dan aktivasi (tentu saja kita bisa menambahkannya sesuai kebutuhan)
    def __init__(self, units, activation=None):
        super().__init__()
        self.units = units
        self.activation= activation

    # Pada method build kita inisialisasi weight dan bias, fungsi build hanya dijalankan satu kali
    # Jadi weight dan bias tetap akan ter-update saat back propagation
    def build(self, X_shape):
        self.weight = self.add_weight(name='weight',
            shape=[X_shape[-1], self.units],
            initializer=tf.random_normal_initializer())
        self.bias = self.add_weight(
            name='bias', shape=[self.units],
            initializer=tf.zeros_initializer())

    # Pada fungsi call kita melakukan operasi (input * weight + bias lalu aktivasi)
    def call(self, X):
        linear = tf.matmul(X, self.weight) + self.bias

        if self.activation is None:
            return linear
        
        return self.activation(linear)

In [24]:
temp_dense= MyDense(5)
temp_dense(X)

temp_dense.get_weights()

[array([[-0.09609327,  0.0039675 , -0.00946576,  0.05620673,  0.01331143],
        [-0.02206794,  0.04316601, -0.10465799,  0.02701032, -0.00904296],
        [-0.00410918,  0.02375493, -0.02565541, -0.01757707, -0.06320279],
        [ 0.0036046 ,  0.10722344,  0.03886317, -0.06740361, -0.01180181]],
       dtype=float32), array([0., 0., 0., 0., 0.], dtype=float32)]

Selanjutnya membuat model dengan 2 layer MyDense yang telah dibuat

In [25]:
net= keras.Sequential([
    keras.layers.Flatten(),
    MyDense(128, tf.nn.relu), 
    MyDense(64, tf.nn.relu), 
    MyDense(32, tf.nn.relu), 
    MyDense(10, tf.nn.softmax)
])

net.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

Kita akan mengetes model tersebut pada dataset mnist

In [26]:
(x_train, y_train), (x_test, y_test)= keras.datasets.mnist.load_data()
x_train, x_test= x_train/255, x_test/255

11501568/11490434 [==============================] - 0s 0us/step


In [27]:
net.fit(x_train, y_train, epochs=3, batch_size= 128)

Epoch 1/3
469/469 [==============================] - 4s 6ms/step - loss: 0.5412 - accuracy: 0.8410
Epoch 2/3
469/469 [==============================] - 3s 6ms/step - loss: 0.2023 - accuracy: 0.9420
Epoch 3/3
469/469 [==============================] - 3s 7ms/step - loss: 0.1433 - accuracy: 0.9579


In [28]:
net.evaluate(x_test, y_test)

313/313 [==============================] - 1s 2ms/step - loss: 0.1254 - accuracy: 0.9620


[0.1253914088010788, 0.9620000123977661]

**5. File I/O**

**Save / Load Tensor**

Untuk tensor, kita bisa menggunakan numpy.save(namaFile.npy, tensor) untuk menyimpan tensor dan numpy.load(namaFile.npy) untuk memuatnya.

In [29]:
save_x= tf.constant(['a', 'b', 'c'])
save_x

<tf.Tensor: shape=(3,), dtype=string, numpy=array([b'a', b'b', b'c'], dtype=object)>

Menyimpan tensor

In [30]:
import numpy as np

np.save('save-x.npy', save_x)

Meload tensor

In [31]:
temp_x= np.load('save-x.npy', allow_pickle=True)
temp_x

array([b'a', b'b', b'c'], dtype=object)

**Save / Load Model Parameter**

Misal model kita seperti ini

In [32]:
class MLP(tf.keras.Model):
    def __init__(self):
        super().__init__()
        self.flatten = tf.keras.layers.Flatten()
        self.hidden = tf.keras.layers.Dense(units=256, activation=tf.nn.relu)
        self.out = tf.keras.layers.Dense(units=10)

    def call(self, inputs):
        x = self.flatten(inputs)
        x = self.hidden(x)
        return self.out(x)

net= MLP()

Lalu kita melakukan feed forward pada model

In [33]:
Y1= net(X)
Y1

<tf.Tensor: shape=(2, 10), dtype=float32, numpy=
array([[-0.05631888,  0.09221004, -0.0774105 ,  0.0601402 ,  0.1515707 ,
        -0.08037113, -0.14321855, -0.02205836, -0.12156422, -0.02067745],
       [-0.10202052,  0.06543152, -0.06169759,  0.11037045,  0.00150129,
        -0.10857888, -0.13019027, -0.04600454, -0.18177006, -0.12013099]],
      dtype=float32)>

Sekarang kita akan menyimpan parameter model dengan fungsi save_weights('nama_weight.params')

In [34]:
net.save_weights('mlp.params')

Sekarang kita akan membuat model dengan arsitektur yang sama tapi tidak dilakukan feed forward, sebaliknya parameternya diambil/diload dari file parameter yang telah di-save sebelumnya

In [35]:
net_2= MLP()
net_2.load_weights('mlp.params')

# Feed forward
Y2= net_2(X)

Sekarang kita akan membandingkan variabel Y1 dengan Y2

In [36]:
Y1==Y2

<tf.Tensor: shape=(2, 10), dtype=bool, numpy=
array([[ True,  True,  True,  True,  True,  True,  True,  True,  True,
         True],
       [ True,  True,  True,  True,  True,  True,  True,  True,  True,
         True]])>

Hasilnya adalah sama

**6. GPUs**

Mengecek ketersediaan GPU

In [37]:
!nvidia-smi

NVIDIA-SMI has failed because it couldn't communicate with the NVIDIA driver. Make sure that the latest NVIDIA driver is installed and running.



Kita dapat menentukan device, seperti CPU dan GPU, untuk penyimpanan dan komputasi. Secara default, tensor dibuat di memori utama dan menggunakan CPU untuk komputasi.

In [38]:
tf.device('/CPU:0'), tf.device('/GPU:0'), tf.device('/GPU:1')

(<tensorflow.python.eager.context._EagerDeviceContext at 0x7f4debaa3190>,
 <tensorflow.python.eager.context._EagerDeviceContext at 0x7f4debaa3f50>)

Kita bisa menjalankan query untuk mendapatkan berapa banyak GPU dalam sistem

In [39]:
len(tf.config.experimental.list_physical_devices('GPU'))

0

**Tensor dan GPU**

Secara default, tensor dibuat dalam CPU, untuk melihat di device mana suatu tensor berada kita bisa menjalankan fungsi seperti ini

In [40]:
x = tf.constant([1, 2, 3])
x.device

'/job:localhost/replica:0/task:0/device:CPU:0'

Outputnya mengatakan GPU karena saya menggunakan google colabs dan telah mengatur runtime pada saat penulisan kode, namun secara default, kalau tidak didefinisikan, akan mengeluarkan output CPU

**Menyetel penggunaan GPU**

Misal kita ingin menggunakan GPU pertama atau GPU:0

In [41]:
tf.device('GPU:0')

In [42]:
with tf.device('GPU:0'):
    test = tf.ones((2, 3))

test.device

'/job:localhost/replica:0/task:0/device:CPU:0'

Hati-hati saat pemilihan GPU terutama jika kita memiliki lebih dari satu GPU dalam satu komputer, karena akan terjadi error jika kita melakukan operasi yang melibatkan 2 tensor yang berada di GPU yang berbeda